In [ ]:
%pip install datasets

In [ ]:
from datasets import load_dataset


In [3]:
# Load the McGill-NLP/medal dataset in streaming mode
dataset = load_dataset('McGill-NLP/medal', split='train', streaming=True)

In [4]:
# Check if the dataset is defined and inspect a sample
print(type(dataset))  # Should print something like 'datasets.iterable_dataset.IterableDataset'
example = next(iter(dataset))  # Get the first example from the stream
print(example)  # Print the first example

<class 'datasets.iterable_dataset.IterableDataset'>
{'abstract_id': 14145090, 'text': 'velvet antlers vas are commonly used in traditional chinese medicine and invigorant and contain many PET components for health promotion the velvet antler peptide svap is one of active components in vas based on structural study the svap interacts with tgfÎ² receptors and disrupts the tgfÎ² pathway we hypothesized that svap prevents cardiac fibrosis from pressure overload by blocking tgfÎ² signaling SDRs underwent TAC tac or a sham operation T3 one month rats received either svap mgkgday or vehicle for an additional one month tac surgery induced significant cardiac dysfunction FB activation and fibrosis these effects were improved by treatment with svap in the heart tissue tac remarkably increased the expression of tgfÎ² and connective tissue growth factor ctgf ROS species C2 and the phosphorylation C2 of smad and ERK kinases erk svap inhibited the increases in reactive oxygen species C2 ctgf express

In [5]:
# Initialize variables to track null values
any_null = False
null_columns = {}

In [6]:
# Iterate through the streamed dataset
for example in dataset:
    # Check for null values in the current row
    for key, value in example.items():
        if value is None:
            any_null = True
            if key in null_columns:
                null_columns[key] += 1
            else:
                null_columns[key] = 1


In [5]:
# Print whether any null values were found
print("Any null values in the dataset:", any_null)

# If there are null values, print which columns have them and how many
if any_null:
    print("Columns with null values and their counts:")
    for column, count in null_columns.items():
        print(f"{column}: {count} null values")
else:
    print("No null values found in the dataset.")


Any null values in the dataset: False
No null values found in the dataset.


In [11]:
# Initialize variables to count rows and columns
row_count = 0
column_set = set()

# Iterate through the streamed dataset
for example in dataset:
    row_count += 1
    column_set.update(example.keys())  # Add columns to the set (unique columns)

# Convert the set to a list to get the number of unique columns
column_count = len(column_set)

# Print the results
print(f"Total rows in the dataset: {row_count}")
print(f"Total columns in the dataset: {column_count}")


Total rows in the dataset: 3000000
Total columns in the dataset: 4


In [13]:
# checking randomly for a value (in this case third record value)
# Initialize a counter
counter = 0
third_record = None

# Iterate through the dataset to get the 3rd record
for example in dataset:
    counter += 1
    if counter == 3:
        third_record = example
        break

# Print the 3rd record
if third_record:
    print("3rd Record:")
    print(third_record)
else:
    print("The dataset has fewer than 3 records.")


3rd Record:
{'abstract_id': 8625554, 'text': 'ceftobiprole bpr is an investigational cephalosporin with activity against staphylococcus aureus including methicillinresistant s aureus mrsa strains the pharmacodynamic pd profile of bpr against s aureus strains with a variety of susceptibility phenotypes in an immunocompromised murine pneumonia model was characterized the bpr mics of the test isolates ranged from to mugml pharmacokinetic pk studies were conducted with infected neutropenic balbc mice and the bpr concentrations were measured in plasma epithelial lining fluid elf and lung tissue pd studies with these mice were undertaken with eight s aureus isolates two MSSA strains three hospitalacquired mrsa strains and three CA mrsa strains subcutaneous bpr doses of to mgkg of body weightday were administered and the NC in the number of log cfuml in lungs was evaluated after h of therapy the pd profile was characterized by using the free drug exposures f determined from the following para

In [12]:
# Get the first example from the dataset to determine column names
first_example = next(iter(dataset))

# Print the column names
column_names = list(first_example.keys())
print("Column names:", column_names)


Column names: ['abstract_id', 'text', 'location', 'label']


In [ ]:
# Example code to check for null values in each column
null_counts = {column: 0 for column in ['abstract_id', 'text', 'location', 'label']}
for example in dataset:
    for column in null_counts:
        if example[column] is None:
            null_counts[column] += 1
print("Null values per column:", null_counts)


In [ ]:
# Preview the first few records
preview_count = 5  # Number of records to preview
preview_data = [next(iter(dataset)) for _ in range(preview_count)]
for record in preview_data:
    print(record)
